In [9]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [10]:
filename = "../../data/02/ABox.ttl"

In [11]:
text1 = '''CQ_2.1
Return the risk summaries and the agents of deterioration of the risks assigned to the "Battistero degli Ariani".
'''

query1 = '''
PREFIX : <http://purl.org/sirius/ontology/02/data/>
PREFIX hero: <http://purl.org/sirius/ontology/hero/>

SELECT ?risk ?risk_summary ?agent_of_deterioration
WHERE {
    ?risk_assessment a hero:RiskAssessment ;
                    hero:assignsRiskTo :battistero-degli-ariani ;
                    hero:assigns ?risk .
    ?risk a hero:Risk ;
        hero:hasDescription ?risk_summary ;
        hero:isDeterminedBy ?agent_of_deterioration .
}
'''

In [12]:
text2 = '''CQ_2.2
Return the hazards originating the risk that are related to the agent of deterioration "Physical forces".
'''

query2 = '''
PREFIX hero: <http://purl.org/sirius/ontology/hero/>

SELECT ?risk ?hazard_type
WHERE {
    ?risk a hero:Risk ;
            hero:resultsFrom ?hazard ;
            hero:isDeterminedBy hero:physical-forces .
    ?hazard a hero:Hazard ;
            hero:hasHazardType ?hazard_type .
}
'''

In [13]:
text3 = '''CQ_2.3
Return the adverse effects of the risk that is related to the agents of deterioration "Pests", "Water" and "Fire".
'''

query3 = '''
PREFIX hero: <http://purl.org/sirius/ontology/hero/>

SELECT ?risk ?agent_of_deterioration ?adverse_effect_type
WHERE {
    ?risk a hero:Risk ;
            hero:entails ?adverse_effect ;
            hero:isDeterminedBy ?agent_of_deterioration .
    ?adverse_effect a hero:AdverseEffect ;
                    hero:hasAdverseEffectType ?adverse_effect_type .
    FILTER (
    ?agent_of_deterioration = hero:pests || 
    ?agent_of_deterioration = hero:water || 
    ?agent_of_deterioration = hero:fire
    )
}
'''

In [14]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3)]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_2.1
Return the risk summaries and the agents of deterioration of the risks assigned to the "Battistero degli Ariani".

+-------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------+
| risk                                                                    | risk_summary                                                                                                                                                  | agent_of_deterioration                                   |
|-------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------